# 다중인덱스 , 프레임 병합

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

import warnings
warnings.filterwarnings(action = 'ignore') 

from datetime import date, datetime, timedelta
from dateutil.parser import parse

print("numpy version:",np.__version__) 
print("pandas version:",pd.__version__)

numpy version: 1.20.1
pandas version: 1.2.4


In [2]:
def seriesInfo(ary):
    print('data \n', ary)
    print('index : ', type(ary.index), ary.index)
    print('value : ', type(ary.values), ary.values)
    
def frmInfo(df):
    display('DataFrame',df)
    print('shape : ', df.shape) # 형태
    print('size : ' , df.size) # 전체 개수
    print('ndim : ' , df.ndim)
    print('index(row) : ', df.index, type(df.index)) # row index
    print('columns : ', df.columns, type(df.columns))
    print('values : \n', df.values, type(df.values))

## 다중인덱스 (열)

In [4]:
np.random.seed(100)
multi_frm=pd.DataFrame(np.round( np.random.randn(5,4),2),
                      columns=[['Grop01','Grop01','Grop02','Grop02'],
                               ['col01','col02','col1','col02']])
multi_frm

Grop01       Grop02      
   col01 col02   col1 col02
0  -1.75  0.34   1.15 -0.25
1   0.98  0.51   0.22 -1.07
2  -0.19  0.26  -0.46  0.44
3  -0.58  0.82   0.67 -0.10
4  -0.53  1.03  -0.44 -1.12

- 열 인덱스에 이름을 부여하여 사용의 편리성을 높일 수 있다.

In [5]:
multi_frm.columns # 튜플형태로 나옴.

MultiIndex([('Grop01', 'col01'),
            ('Grop01', 'col02'),
            ('Grop02',  'col1'),
            ('Grop02', 'col02')],
           )

In [6]:
multi_frm.columns.names=['Grpidx','Colidx'] # 다중인덱스에 이름 부여
multi_frm.columns

MultiIndex([('Grop01', 'col01'),
            ('Grop01', 'col02'),
            ('Grop02',  'col1'),
            ('Grop02', 'col02')],
           names=['Grpidx', 'Colidx'])

In [7]:
multi_frm

Grpidx Grop01       Grop02      
Colidx  col01 col02   col1 col02
0       -1.75  0.34   1.15 -0.25
1        0.98  0.51   0.22 -1.07
2       -0.19  0.26  -0.46  0.44
3       -0.58  0.82   0.67 -0.10
4       -0.53  1.03  -0.44 -1.12

## 다중인덱스 (행)

In [9]:
np.random.seed(100)
multi_frm=pd.DataFrame(np.round( np.random.randn(6,4),2),
                      columns=[['Grop01','Grop01','Grop02','Grop02'],
                               ['col01','col02','col1','col02']],
                      index = [["M","M","M","F","F","F"],["id_"+str(i+1) for i in range(6)]])
multi_frm.columns.names=['Grpidx','Colidx']
multi_frm.index.names=['gender','idx']
multi_frm

Grpidx      Grop01       Grop02      
Colidx       col01 col02   col1 col02
gender idx                           
M      id_1  -1.75  0.34   1.15 -0.25
       id_2   0.98  0.51   0.22 -1.07
       id_3  -0.19  0.26  -0.46  0.44
F      id_4  -0.58  0.82   0.67 -0.10
       id_5  -0.53  1.03  -0.44 -1.12
       id_6   1.62  1.54  -0.25 -0.84

## 인덱스를 바꿀 수 있다
### stack이 (위에서 아래로 길게, 높게) 쌓는 것이면, unstack은 쌓은 것을 옆으로 늘어놓는것(왼쪽에서 오른쪽으로 넓게)

- stack() : 열 → 행
    - pd.DataFrame.stack(level=-1, dropna=True)
    - stack("칼럼이름"|칼럼의 정수인덱스)
    - dropna=True:결측값을 제거, dropna=False : 결측값을 NaN으로 유지
- unstack() : 행 → 열
    - pd.DataFrame.unstack(level=-1, fill_value=None)
    - unstack("행이름"|행의 정수 인덱스)

In [10]:
multi_frm.stack('Colidx') # Colidx를 행인덱스로! 

Grpidx              Grop01  Grop02
gender idx  Colidx                
M      id_1 col01    -1.75     NaN
            col02     0.34   -0.25
            col1       NaN    1.15
       id_2 col01     0.98     NaN
            col02     0.51   -1.07
            col1       NaN    0.22
       id_3 col01    -0.19     NaN
            col02     0.26    0.44
            col1       NaN   -0.46
F      id_4 col01    -0.58     NaN
            col02     0.82   -0.10
            col1       NaN    0.67
       id_5 col01    -0.53     NaN
            col02     1.03   -1.12
            col1       NaN   -0.44
       id_6 col01     1.62     NaN
            col02     1.54   -0.84
            col1       NaN   -0.25

In [13]:
multi_frm.unstack()

Grpidx Grop01                                                        ...  \
Colidx  col01                               col02                    ...   
idx      id_1  id_2  id_3  id_4  id_5  id_6  id_1  id_2  id_3  id_4  ...   
gender                                                               ...   
F         NaN   NaN   NaN -0.58 -0.53  1.62   NaN   NaN   NaN  0.82  ...   
M       -1.75  0.98 -0.19   NaN   NaN   NaN  0.34  0.51  0.26   NaN  ...   

Grpidx Grop02                                                       
Colidx   col1                   col02                               
idx      id_3  id_4  id_5  id_6  id_1  id_2  id_3 id_4  id_5  id_6  
gender                                                              
F         NaN  0.67 -0.44 -0.25   NaN   NaN   NaN -0.1 -1.12 -0.84  
M       -0.46   NaN   NaN   NaN -0.25 -1.07  0.44  NaN   NaN   NaN  

[2 rows x 24 columns]

## 다중인덱스의 경우 인덱싱은 어떻게 할까?
### 튜플형식으로 접근하면 된다!

In [21]:
multi_frm

Grpidx      Grop01       Grop02      
Colidx       col01 col02   col1 col02
gender idx                           
M      id_1  -1.75  0.34   1.15 -0.25
       id_2   0.98  0.51   0.22 -1.07
       id_3  -0.19  0.26  -0.46  0.44
F      id_4  -0.58  0.82   0.67 -0.10
       id_5  -0.53  1.03  -0.44 -1.12
       id_6   1.62  1.54  -0.25 -0.84

In [22]:
multi_frm[[('Grop01','col01'),('Grop01','col02')]]

Grpidx      Grop01      
Colidx       col01 col02
gender idx              
M      id_1  -1.75  0.34
       id_2   0.98  0.51
       id_3  -0.19  0.26
F      id_4  -0.58  0.82
       id_5  -0.53  1.03
       id_6   1.62  1.54

In [39]:
print(multi_frm[('Grop02','col1')][0])
print(multi_frm.loc[('M','id_1'),('Grop02','col1')])
display(multi_frm.loc[[('M','id_1')] , :])

display(multi_frm.loc['M'])

1.15
1.15


Grpidx      Grop01       Grop02      
Colidx       col01 col02   col1 col02
gender idx                           
M      id_1  -1.75  0.34   1.15 -0.25

Grpidx Grop01       Grop02      
Colidx  col01 col02   col1 col02
idx                             
id_1    -1.75  0.34   1.15 -0.25
id_2     0.98  0.51   0.22 -1.07
id_3    -0.19  0.26  -0.46  0.44

## 다중인덱스의 정렬
- level 속성을 이용해서 기준을 정의해야 한다.
    - 정렬할 level을 선택한다.
- axis를 활용해서 축을 지정해줘야 한다.
    - 메소드를 적용할 축을 지정한다

In [40]:
multi_frm.sort_index(level=0) # gender를 기준으로 오름차순 정렬

Grpidx      Grop01       Grop02      
Colidx       col01 col02   col1 col02
gender idx                           
F      id_4  -0.58  0.82   0.67 -0.10
       id_5  -0.53  1.03  -0.44 -1.12
       id_6   1.62  1.54  -0.25 -0.84
M      id_1  -1.75  0.34   1.15 -0.25
       id_2   0.98  0.51   0.22 -1.07
       id_3  -0.19  0.26  -0.46  0.44

In [52]:
multi_frm.sort_index(level=0,axis=1)

Grpidx      Grop01       Grop02      
Colidx       col01 col02  col02  col1
gender idx                           
M      id_1  -1.75  0.34  -0.25  1.15
       id_2   0.98  0.51  -1.07  0.22
       id_3  -0.19  0.26   0.44 -0.46
F      id_4  -0.58  0.82  -0.10  0.67
       id_5  -0.53  1.03  -1.12 -0.44
       id_6   1.62  1.54  -0.84 -0.25

## 프레임 반환
- pd.merge : 공통의 열이 있어야 함! (열을 기준으로 병합, 옵션을 줘서 innerjoin, 인덱스 병합도 가능) 
- concat   : 연결
- frm.join : 행 index를 기준으로 병합한다.

In [54]:
data1={
    '학번':[1,2,3,4],
    '이름':['A',"B","C","D"],
    '학년':[2,4,1,3]
}
data2={
    '학번':[1,2,4,5],
    '학과':['CS',"AI","AI","CS"],
    '학점':[2.4,4.5,1.7,3.9]

}

In [55]:
stu_frm = pd.DataFrame(data1)
major_frm = pd.DataFrame(data2)
display(stu_frm)
print()
display(major_frm)

,학번,이름,학년
0,1,A,2
1,2,B,4
2,3,C,1
3,4,D,3


,학번,학과,학점
0,1,CS,2.4
1,2,AI,4.5
2,4,AI,1.7
3,5,CS,3.9


In [56]:
# merge함수로 데이터프레임 병합
pd.merge(stu_frm,major_frm) # sql의 innerjoin이라고 생각하면 됨(공통되는 요소만 병합되어 리턴)

,학번,이름,학년,학과,학점
0,1,A,2,CS,2.4
1,2,B,4,AI,4.5
2,4,D,3,AI,1.7


In [57]:
# inner join말고 outer join으로 merge함수 사용해서 병합!
pd.merge(stu_frm, major_frm, how='outer') # how= left | right | outer | inner로  어떤 join할건지 지정해줄 수 있다!

,학번,이름,학년,학과,학점
0,1,A,2.0,CS,2.4
1,2,B,4.0,AI,4.5
2,3,C,1.0,NaN,NaN
3,4,D,3.0,AI,1.7
4,5,NaN,NaN,CS,3.9


### 칼럼의 인덱스가 다른 경우 merge (동일한 컬럼이 없을 경우)

In [59]:
data1={
    '학번':[1,2,3,4],
    '이름':['A',"B","C","D"],
    '학년':[2,4,1,3]
}
data2={
    '과목코드':[1,2,4,5],
    '학과':['CS',"AI","AI","CS"],
    '학점':[2.4,4.5,1.7,3.9]

}
stu_frm = pd.DataFrame(data1)
major_frm = pd.DataFrame(data2)
display(stu_frm)
print()
display(major_frm)

,학번,이름,학년
0,1,A,2
1,2,B,4
2,3,C,1
3,4,D,3


,과목코드,학과,학점
0,1,CS,2.4
1,2,AI,4.5
2,4,AI,1.7
3,5,CS,3.9


In [62]:
# 공통되는 열이 없는 경우!
pd.merge(stu_frm, major_frm, how='inner', left_on='학번' , right_on='과목코드')

,학번,이름,학년,과목코드,학과,학점
0,1,A,2,1,CS,2.4
1,2,B,4,2,AI,4.5
2,4,D,3,4,AI,1.7


In [63]:
import seaborn as sns
datasets = sns.load_dataset('iris')
print('type - ')
print( type(datasets))

type - 
<class 'pandas.core.frame.DataFrame'>


In [64]:
datasets.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


- 열 인덱스의 값이 같은 데이터로 이루어져 있는 경우?

In [82]:
data01={
    'species':['setosa','virginica','virginica','versicolor'],
    'sepal_length':[5.1, 4.9, 4.7, 4.6]
}
data02={
    'species':['setosa','setosa','virginica','virginica'],
    'sepal_width':[3.5, 3.0, 3.2, 3.1]
}

iris01_frm=pd.DataFrame(data01)
iris02_frm=pd.DataFrame(data02)
display(iris01_frm)
print()
display(iris02_frm)

,species,sepal_length
0,setosa,5.1
1,virginica,4.9
2,virginica,4.7
3,versicolor,4.6


,species,sepal_width
0,setosa,3.5
1,setosa,3.0
2,virginica,3.2
3,virginica,3.1


- on : 중복되는 키가 존재할경우 어느 키를 기준으로 merge할지를 결정하게 된다.

In [83]:
pd.merge(iris01_frm, iris02_frm, on='species') # iris01_frm을 기준으로 경우의 수를 포함한 dataframe 리턴

,species,sepal_length,sepal_width
0,setosa,5.1,3.5
1,setosa,5.1,3.0
2,virginica,4.9,3.2
3,virginica,4.9,3.1
4,virginica,4.7,3.2
5,virginica,4.7,3.1


In [84]:
data01={
    'species':['setosa','virginica','virginica','versicolor'],
    'sepal_length':[5.1, 4.9, 4.7, 4.6],
    'sepal_width':[4.1, 3.9, 3.7, 3.6]
}
data02={
    'species':['setosa','setosa','virginica','virginica'],
    'sepal_length':[3.5, 3.0, 3.2, 3.1]
}

iris01_frm=pd.DataFrame(data01)
iris02_frm=pd.DataFrame(data02)

In [85]:
pd.merge(iris01_frm, iris02_frm, on ='species') # x, y로 나눠서 경우의 수를 포함한 dataframe을 리턴

,species,sepal_length_x,sepal_width,sepal_length_y
0,setosa,5.1,4.1,3.5
1,setosa,5.1,4.1,3.0
2,virginica,4.9,3.9,3.2
3,virginica,4.9,3.9,3.1
4,virginica,4.7,3.7,3.2
5,virginica,4.7,3.7,3.1


### 칼럼 인덱스가 아닌 인덱스를 기준으로 병합한다면?
- left_index, right_index

In [87]:
pop01_frm = pd.DataFrame({
    'city':['seoul','seoul','seoul','seongnam','seongnam'],
    'year':[2010,2050,2020,2018,2015],
    'pop' :[1234567,2345678,3456789,4567890,5678901]
})
pop01_frm

,city,year,pop
0,seoul,2010,1234567
1,seoul,2050,2345678
2,seoul,2020,3456789
3,seongnam,2018,4567890
4,seongnam,2015,5678901


In [91]:
pop02_frm=pd.DataFrame(np.arange(12).reshape(6,2),
                      columns=['col01','col02'],
                      index=[['seoul','seoul','seongnam','seongnam','seongnam','seongnam'],[2010,2050,2020,2018,2015,2020]])
pop02_frm

col01  col02
seoul    2010      0      1
         2050      2      3
seongnam 2020      4      5
         2018      6      7
         2015      8      9
         2020     10     11

In [94]:
pd.merge(pop01_frm, pop02_frm, right_index=True, left_on=['city', 'year'])
# 같은 인덱스가 없어서 right_index=True로 줘서 기준을 정하고, city, year을 기준으로 일치하는 값만 가져옴.

,city,year,pop,col01,col02
0,seoul,2010,1234567,0,1
1,seoul,2050,2345678,2,3
3,seongnam,2018,4567890,6,7
4,seongnam,2015,5678901,8,9


In [99]:
data1 = {
    '이름' : ['A' , 'B' , 'C' , 'D'] , 
    '학년' : [2,4,1,3]
}
data2 = {
    '학과' : ['CS' , 'AI' , 'AI' , 'CS'] , 
    '학점' : [2.4 , 4.5 , 1.7 , 3.9]
}
stu_frm   = pd.DataFrame(data1,index=[1,2,3,4])
major_frm = pd.DataFrame(data2,index=[1,2,4,5])
display(stu_frm)
print()
display(major_frm)

,이름,학년
1,A,2
2,B,4
3,C,1
4,D,3


,학과,학점
1,CS,2.4
2,AI,4.5
4,AI,1.7
5,CS,3.9


In [100]:
pd.merge(stu_frm,major_frm,right_index=True, left_index=True) # 옵션주지 않으면 열인덱스를 기준으로 병합해서 오류남
# index가 동일한것만 매칭해서 병합해줌.

,이름,학년,학과,학점
1,A,2,CS,2.4
2,B,4,AI,4.5
4,D,3,AI,1.7


## join은 행인덱스를 기준으로 병합한다.

In [102]:
stu_frm.join(major_frm,how='inner') 

,이름,학년,학과,학점
1,A,2,CS,2.4
2,B,4,AI,4.5
4,D,3,AI,1.7


## concat은 열 인덱스 또는 행 인덱스를 기준으로 하지 않고 단순하게 데이터를 연결해준다. 

In [104]:
s01 = pd.Series([0,1])
s02 = pd.Series([2,3,4])
pd.concat([s01,s02]) # 단순하게 연결해줌

0    0
1    1
0    2
1    3
2    4
dtype: int64

In [109]:
concat01_frm=pd.DataFrame(np.arange(6).reshape(3,2),
                       index=['a','b','c'],
                       columns=['col01','col02'])
concat02_frm=pd.DataFrame(np.arange(4).reshape(2,2),
                       index=['a','c'],
                       columns=['col03','col04'])

display(concat01_frm)
print()
display(concat02_frm)


,col01,col02
a,0,1
b,2,3
c,4,5


,col03,col04
a,0,1
c,2,3


In [112]:
pd.concat([concat01_frm,concat02_frm],axis=1 ) # axis=1이라 열에 대한 연결

,col01,col02,col03,col04
a,0,1,0.0,1.0
b,2,3,NaN,NaN
c,4,5,2.0,3.0


In [116]:
stock01=pd.read_excel('C:/Users/whgud/data/stock_price.xlsx')
stock01

,id,stock_name,value,price
0,128940,한미약품,59385.666667,421000
1,130960,CJ E&M,58540.666667,98900
2,138250,엔에스쇼핑,14558.666667,13200
3,139480,이마트,239230.833333,254500
4,142280,녹십자엠에스,468.833333,10200
5,145990,삼양사,82750.000000,82000
6,185750,종근당,40293.666667,100500
7,192400,쿠쿠홀딩스,179204.666667,177500
8,199800,툴젠,-2514.333333,115400
9,204210,모두투어리츠,3093.333333,3475


In [117]:
stock02=pd.read_excel('C:/Users/whgud/data/stock_valuation.xlsx')
stock02

,id,name,eps,bps,per,pbr
0,130960,CJ E&M,6301.333333,54068,15.695091,1.829178
1,136480,하림,274.166667,3551,11.489362,0.887074
2,138040,메리츠금융지주,2122.333333,14894,6.313806,0.899691
3,139480,이마트,18268.166667,295780,13.931338,0.860437
4,145990,삼양사,5741.000000,108090,14.283226,0.758627
5,161390,한국타이어,5648.500000,51341,7.453306,0.820007
6,181710,NHN엔터테인먼트,2110.166667,78434,30.755864,0.827447
7,185750,종근당,3990.333333,40684,25.185866,2.470259
8,204210,모두투어리츠,85.166667,5335,40.802348,0.651359
9,207940,삼성바이오로직스,4644.166667,60099,89.790059,6.938551


In [123]:
# merge를 사용해서 stock01,stock02 데이터프레임 병합하기
pd.merge(stock01,stock02) # 일치하는 것만!

,id,stock_name,value,price,name,eps,bps,per,pbr
0,130960,CJ E&M,58540.666667,98900,CJ E&M,6301.333333,54068,15.695091,1.829178
1,139480,이마트,239230.833333,254500,이마트,18268.166667,295780,13.931338,0.860437
2,145990,삼양사,82750.000000,82000,삼양사,5741.000000,108090,14.283226,0.758627
3,185750,종근당,40293.666667,100500,종근당,3990.333333,40684,25.185866,2.470259
4,204210,모두투어리츠,3093.333333,3475,모두투어리츠,85.166667,5335,40.802348,0.651359


In [137]:
# join사용해서 stock01,stock02 데이터프레임 병합하기
stock01.join(stock02) # 'id'라는 칼럼이 중복되기 때문에 에러 발생

ValueError: columns overlap but no suffix specified: Index(['id'], dtype='object')

In [138]:
# 해결법 : 두 데이터프레임의 'id' 칼럼이름을 변경해줘야 한다.
# lsuffix(Left suffix) 옵션과 rsuffix(Right Suffix)옵션을 활용하여 id 이름 변경해주기!
stock01.join(stock02, lsuffix="id_stock01",rsuffix="id_stock02")

,idid_stock01,stock_name,value,price,idid_stock02,name,eps,bps,per,pbr
0,128940,한미약품,59385.666667,421000,130960,CJ E&M,6301.333333,54068,15.695091,1.829178
1,130960,CJ E&M,58540.666667,98900,136480,하림,274.166667,3551,11.489362,0.887074
2,138250,엔에스쇼핑,14558.666667,13200,138040,메리츠금융지주,2122.333333,14894,6.313806,0.899691
3,139480,이마트,239230.833333,254500,139480,이마트,18268.166667,295780,13.931338,0.860437
4,142280,녹십자엠에스,468.833333,10200,145990,삼양사,5741.000000,108090,14.283226,0.758627
5,145990,삼양사,82750.000000,82000,161390,한국타이어,5648.500000,51341,7.453306,0.820007
6,185750,종근당,40293.666667,100500,181710,NHN엔터테인먼트,2110.166667,78434,30.755864,0.827447
7,192400,쿠쿠홀딩스,179204.666667,177500,185750,종근당,3990.333333,40684,25.185866,2.470259
8,199800,툴젠,-2514.333333,115400,204210,모두투어리츠,85.166667,5335,40.802348,0.651359
9,204210,모두투어리츠,3093.333333,3475,207940,삼성바이오로직스,4644.166667,60099,89.790059,6.938551


In [134]:
# concat사용해서 stock01,stock02 데이터프레임 병합하기
pd.concat([stock01,stock02],axis=1)

,id,stock_name,value,price,id,name,eps,bps,per,pbr
0,128940,한미약품,59385.666667,421000,130960,CJ E&M,6301.333333,54068,15.695091,1.829178
1,130960,CJ E&M,58540.666667,98900,136480,하림,274.166667,3551,11.489362,0.887074
2,138250,엔에스쇼핑,14558.666667,13200,138040,메리츠금융지주,2122.333333,14894,6.313806,0.899691
3,139480,이마트,239230.833333,254500,139480,이마트,18268.166667,295780,13.931338,0.860437
4,142280,녹십자엠에스,468.833333,10200,145990,삼양사,5741.000000,108090,14.283226,0.758627
5,145990,삼양사,82750.000000,82000,161390,한국타이어,5648.500000,51341,7.453306,0.820007
6,185750,종근당,40293.666667,100500,181710,NHN엔터테인먼트,2110.166667,78434,30.755864,0.827447
7,192400,쿠쿠홀딩스,179204.666667,177500,185750,종근당,3990.333333,40684,25.185866,2.470259
8,199800,툴젠,-2514.333333,115400,204210,모두투어리츠,85.166667,5335,40.802348,0.651359
9,204210,모두투어리츠,3093.333333,3475,207940,삼성바이오로직스,4644.166667,60099,89.790059,6.938551


In [139]:
stock01 # 병합한 결과들은 새로운 데이터프레임으로 리턴된거라 원본데이터에 반영X

,id,stock_name,value,price
0,128940,한미약품,59385.666667,421000
1,130960,CJ E&M,58540.666667,98900
2,138250,엔에스쇼핑,14558.666667,13200
3,139480,이마트,239230.833333,254500
4,142280,녹십자엠에스,468.833333,10200
5,145990,삼양사,82750.000000,82000
6,185750,종근당,40293.666667,100500
7,192400,쿠쿠홀딩스,179204.666667,177500
8,199800,툴젠,-2514.333333,115400
9,204210,모두투어리츠,3093.333333,3475


## 그룹만들어주기!
- groupby(열 또는 열리스트 | 행 인덱스)
- 그룹연산 (size, count, mean, median, min, max, sum, std, var, quantile, first, last, aggregate, discribe, transform )
- 분할(split)   : 특정 조건에 따라서 분할
- 적용(apply)   : 집계, 변환, 필터링
- 결합(combine) : 처리된 결과를 하나로 결합

In [140]:
group_frm=pd.DataFrame({
    '학과':['인공지능','데이터분석','인공지능','데이터분석','데이터분석'],
    '학년':[1,2,3,4,2],
    '이름':['A','B','C','D','E'],
    '학점':[2.7, 3.5, 4.5, 3.8, 4.3]
})
group_frm

,학과,학년,이름,학점
0,인공지능,1,A,2.7
1,데이터분석,2,B,3.5
2,인공지능,3,C,4.5
3,데이터분석,4,D,3.8
4,데이터분석,2,E,4.3


In [142]:
# series를 기준으로 groupby하기
dept_series = group_frm['학과'].groupby(group_frm['학과'])
dept_series.groups

{'데이터분석': [1, 3, 4], '인공지능': [0, 2]}

In [143]:
dept_series.get_group('데이터분석')

1    데이터분석
3    데이터분석
4    데이터분석
Name: 학과, dtype: object

In [ ]:
dept_series.get_group('인공지능')

In [144]:
# dataframe을 기준으로 groupby하기
dept_frm_group = group_frm.groupby(group_frm['학과'])
dept_frm_group

In [145]:
dept_frm_group.groups

{'데이터분석': [1, 3, 4], '인공지능': [0, 2]}

In [147]:
# 학과를 그룹으로 수치형 데이터들의 평균이 나옴.
dept_frm_group.mean()

,학년,학점
학과,,
데이터분석,2.666667,3.866667
인공지능,2.000000,3.600000


In [148]:
dept_frm_group.sum()

,학년,학점
학과,,
데이터분석,8,11.6
인공지능,4,7.2


In [149]:
dept_frm_group.agg([np.mean, np.sum])

학년            학점      
           mean sum      mean   sum
학과                                 
데이터분석  2.666667   8  3.866667  11.6
인공지능   2.000000   4  3.600000   7.2

## 다중그룹 만들기

In [150]:
group_frm.groupby(['학과','학년']).groups

{('데이터분석', 2): [1, 4], ('데이터분석', 4): [3], ('인공지능', 1): [0], ('인공지능', 3): [2]}

In [151]:
import seaborn as sns
titanic_datasets = sns.load_dataset('titanic')
iris_datasets    = sns.load_dataset('iris')

In [152]:
titanic_datasets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [153]:
titanic_subset_frm=titanic_datasets[['age','sex','class','fare','survived']]
titanic_subset_frm.head()

,age,sex,class,fare,survived
0,22.0,male,Third,7.2500,0
1,38.0,female,First,71.2833,1
2,26.0,female,Third,7.9250,1
3,35.0,female,First,53.1000,1
4,35.0,male,Third,8.0500,0


In [155]:
titanic_subset_frm.describe() # 수치형데이터만 통계값들.

,age,fare,survived
count,714.000000,891.000000,891.000000
mean,29.699118,32.204208,0.383838
std,14.526497,49.693429,0.486592
min,0.420000,0.000000,0.000000
25%,20.125000,7.910400,0.000000
50%,28.000000,14.454200,0.000000
75%,38.000000,31.000000,1.000000
max,80.000000,512.329200,1.000000


In [156]:
print("승객수 : ",len(titanic_subset_frm))

승객수 :  891


In [161]:
# 선실등급에 따른 그룹
class_grp=titanic_subset_frm.groupby(['class'])

for key ,group in class_grp:
    print('key:',key)
    print('len:',len(group))
    print()
    print(group)
    print("*"*50)

key: First
len: 216

      age     sex  class     fare  survived
1    38.0  female  First  71.2833         1
3    35.0  female  First  53.1000         1
6    54.0    male  First  51.8625         0
11   58.0  female  First  26.5500         1
23   28.0    male  First  35.5000         1
..    ...     ...    ...      ...       ...
871  47.0  female  First  52.5542         1
872  33.0    male  First   5.0000         0
879  56.0  female  First  83.1583         1
887  19.0  female  First  30.0000         1
889  26.0    male  First  30.0000         1

[216 rows x 5 columns]
**************************************************
key: Second
len: 184

      age     sex   class     fare  survived
9    14.0  female  Second  30.0708         1
15   55.0  female  Second  16.0000         1
17    NaN    male  Second  13.0000         1
20   35.0    male  Second  26.0000         0
21   34.0    male  Second  13.0000         1
..    ...     ...     ...      ...       ...
866  27.0  female  Second  13.8583     

In [162]:
class_grp.mean()

,age,fare,survived
class,,,
First,38.233441,84.154687,0.629630
Second,29.877630,20.662183,0.472826
Third,25.140620,13.675550,0.242363


In [163]:
class_grp.agg([np.mean,np.sum])

age                fare              survived     
             mean      sum       mean         sum      mean  sum
class                                                           
First   38.233441  7111.42  84.154687  18177.4125  0.629630  136
Second  29.877630  5168.83  20.662183   3801.8417  0.472826   87
Third   25.140620  8924.92  13.675550   6714.6951  0.242363  119

In [164]:
# class가 Third인 서브셋 만들기!
class_grp_third=class_grp.get_group('Third')
class_grp_third

,age,sex,class,fare,survived
0,22.0,male,Third,7.2500,0
2,26.0,female,Third,7.9250,1
4,35.0,male,Third,8.0500,0
5,NaN,male,Third,8.4583,0
7,2.0,male,Third,21.0750,0
...,...,...,...,...,...
882,22.0,female,Third,10.5167,0
884,25.0,male,Third,7.0500,0
885,39.0,female,Third,29.1250,0
888,NaN,female,Third,23.4500,0


In [166]:
# 선실등급과 성별에 따른 그룹
class_sex_grp=titanic_subset_frm.groupby(['class','sex'])

for key ,group in class_sex_grp:
    print('key:',key)
    print('len:',len(group))
    print()
    print(group)
    print("*"*50)

key: ('First', 'female')
len: 94

      age     sex  class      fare  survived
1    38.0  female  First   71.2833         1
3    35.0  female  First   53.1000         1
11   58.0  female  First   26.5500         1
31    NaN  female  First  146.5208         1
52   49.0  female  First   76.7292         1
..    ...     ...    ...       ...       ...
856  45.0  female  First  164.8667         1
862  48.0  female  First   25.9292         1
871  47.0  female  First   52.5542         1
879  56.0  female  First   83.1583         1
887  19.0  female  First   30.0000         1

[94 rows x 5 columns]
**************************************************
key: ('First', 'male')
len: 122

      age   sex  class      fare  survived
6    54.0  male  First   51.8625         0
23   28.0  male  First   35.5000         1
27   19.0  male  First  263.0000         0
30   40.0  male  First   27.7208         0
34   28.0  male  First   82.1708         0
..    ...   ...    ...       ...       ...
839   NaN  male  F

In [167]:
# First등급인 여자만
class_sex_grp.get_group(('First','female')) # tuple 형태로 넣어주면 OK

,age,sex,class,fare,survived
1,38.0,female,First,71.2833,1
3,35.0,female,First,53.1000,1
11,58.0,female,First,26.5500,1
31,NaN,female,First,146.5208,1
52,49.0,female,First,76.7292,1
...,...,...,...,...,...
856,45.0,female,First,164.8667,1
862,48.0,female,First,25.9292,1
871,47.0,female,First,52.5542,1
879,56.0,female,First,83.1583,1


In [168]:
iris_datasets.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [170]:
# iris data를 종별로 가져온 것.
iris_species_grp=iris_datasets.groupby(['species'])

for key ,group in iris_species_grp:
    print('key:',key)
    print('len:',len(group))
    print()
    print(group)
    print("*"*50)

key: setosa
len: 50

    sepal_length  sepal_width  petal_length  petal_width species
0            5.1          3.5           1.4          0.2  setosa
1            4.9          3.0           1.4          0.2  setosa
2            4.7          3.2           1.3          0.2  setosa
3            4.6          3.1           1.5          0.2  setosa
4            5.0          3.6           1.4          0.2  setosa
5            5.4          3.9           1.7          0.4  setosa
6            4.6          3.4           1.4          0.3  setosa
7            5.0          3.4           1.5          0.2  setosa
8            4.4          2.9           1.4          0.2  setosa
9            4.9          3.1           1.5          0.1  setosa
10           5.4          3.7           1.5          0.2  setosa
11           4.8          3.4           1.6          0.2  setosa
12           4.8          3.0           1.4          0.1  setosa
13           4.3          3.0           1.1          0.1  setosa
14  

In [181]:
# [문제 1] iris 종별(species) 가장 큰 값과 가장 작은 값의 비율을 구한다면?

iris_species_grp=iris_datasets.groupby(['species'])

species_grp=iris_species_grp.agg([np.max, np.min])
species_ratio = iris_species_grp.max()/iris_species_grp.min()
species_ratio 

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,1.348837,1.913043,1.900000,6.000000
versicolor,1.428571,1.700000,1.700000,1.800000
virginica,1.612245,1.727273,1.533333,1.785714


In [ ]:
# [문제 2] iris 종별(species) 가장 큰 petal_length 3개를 구한다면?

In [185]:
# 방법1
iris_datasets.sort_values(by='petal_length',ascending=False)[:3]

,sepal_length,sepal_width,petal_length,petal_width,species
118,7.7,2.6,6.9,2.3,virginica
122,7.7,2.8,6.7,2.0,virginica
117,7.7,3.8,6.7,2.2,virginica


In [186]:
# 방법2
def petal_length_fun(frm):
    return frm.sort_values(by='petal_length',ascending=False)[:3]
iris_species_grp.apply(petal_length_fun)

sepal_length  sepal_width  petal_length  petal_width  \
species                                                                
setosa     24            4.8          3.4           1.9          0.2   
           44            5.1          3.8           1.9          0.4   
           23            5.1          3.3           1.7          0.5   
versicolor 83            6.0          2.7           5.1          1.6   
           77            6.7          3.0           5.0          1.7   
           72            6.3          2.5           4.9          1.5   
virginica  118           7.7          2.6           6.9          2.3   
           117           7.7          3.8           6.7          2.2   
           122           7.7          2.8           6.7          2.0   

                   species  
species                     
setosa     24       setosa  
           44       setosa  
           23       setosa  
versicolor 83   versicolor  
           77   versicolor  
           72   versicolor  
virginica  118   virginica  
           117   virginica  
           122   virginica

In [187]:
iris_species_grp.describe().T

species                setosa  versicolor  virginica
sepal_length count  50.000000   50.000000  50.000000
             mean    5.006000    5.936000   6.588000
             std     0.352490    0.516171   0.635880
             min     4.300000    4.900000   4.900000
             25%     4.800000    5.600000   6.225000
             50%     5.000000    5.900000   6.500000
             75%     5.200000    6.300000   6.900000
             max     5.800000    7.000000   7.900000
sepal_width  count  50.000000   50.000000  50.000000
             mean    3.428000    2.770000   2.974000
             std     0.379064    0.313798   0.322497
             min     2.300000    2.000000   2.200000
             25%     3.200000    2.525000   2.800000
             50%     3.400000    2.800000   3.000000
             75%     3.675000    3.000000   3.175000
             max     4.400000    3.400000   3.800000
petal_length count  50.000000   50.000000  50.000000
             mean    1.462000    4.260000   5.552000
             std     0.173664    0.469911   0.551895
             min     1.000000    3.000000   4.500000
             25%     1.400000    4.000000   5.100000
             50%     1.500000    4.350000   5.550000
             75%     1.575000    4.600000   5.875000
             max     1.900000    5.100000   6.900000
petal_width  count  50.000000   50.000000  50.000000
             mean    0.246000    1.326000   2.026000
             std     0.105386    0.197753   0.274650
             min     0.100000    1.000000   1.400000
             25%     0.200000    1.200000   1.800000
             50%     0.200000    1.300000   2.000000
             75%     0.300000    1.500000   2.300000
             max     0.600000    1.800000   2.500000